In [5]:
from pathlib import Path
from typing import List
import numpy as np
import pandas as pd
from PIL import Image
import requests
from io import BytesIO

from tqdm import tqdm
import os

import requests
from bs4 import BeautifulSoup

In [251]:
class Conv3x3:
      # A Convolution layer using 3x3 filters.

  def __init__(self, num_filters):
    self.num_filters = num_filters

    # filters is a 3d array with dimensions (num_filters, 3, 3)
    # We divide by 9 to reduce the variance of our initial values
    self.filters = np.random.randn(num_filters, 3, 3) / 9

  def iterate_regions(self, image):
    '''
    This actaully produces a region of an image that also captures its DEPTH
    '''
    h, w, _ = image.shape

    for i in range(h - 2):
      for j in range(w - 2):
        im_region = image[i:(i + 3), j:(j + 3)]
        yield im_region, i, j

  def forward(self, input):
    '''
    Performs a forward pass of the conv layer using the given input.
    Returns a 3d numpy array with dimensions (h, w, num_filters).
    - input is a 2d numpy array
    '''
    self.last_input = input

    # change here. Origionally, it didnt take in the "num_filters"(this is just the depth).
    # So now instead of using self.filter which will always be (3,3,8), we use the depth of the given image
    # And to make this simpler/better, we should only throw in the full RGB np.array, instead of grayscale.
    # so for ex) first pass will have k = 3, then k = 3 * 8, ...
    h, w, num_filters = input.shape
    output = np.zeros((h - 2, w - 2, num_filters))
    #print(f"output shape: {output.shape}")

    # we must apply the filters to regions for each filter, one by one.
    for fdx, f in enumerate(self.filters):# new
      for im_region, i, j in self.iterate_regions(input):

        #print(f"imregion: {im_region.shape}")
        #print(f"filter shapeL: {f.shape}")
        output[i, j] = np.sum(im_region * f, axis=(1, 2)) # new    
        #output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

    return output

  def backprop(self, d_L_d_out, learn_rate):
      '''
      Performs a backward pass of the conv layer.
      - d_L_d_out is the loss gradient for this layer's outputs.
      - learn_rate is a float.


      Used ChatGPT to create an output here.... NEED TO TEST
      '''
      d_L_d_filters = np.zeros(self.filters.shape)
      d_L_d_input = np.zeros(self.last_input.shape)  # Gradient w.r.t input

      for im_region, i, j in self.iterate_regions(self.last_input):
          for f in range(self.num_filters):
              d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region
              d_L_d_input[i:i+3, j:j+3] += d_L_d_out[i, j, f] * self.filters[f]

      # Update filters
      self.filters -= learn_rate * d_L_d_filters

      # return the loss gradient for this layer's inputs
      return d_L_d_input

In [185]:
import numpy as np

class MaxPool2:
  # A Max Pooling layer using a pool size of 2.

  def iterate_regions(self, image):
    '''
    Generates non-overlapping 2x2 image regions to pool over.
    - image is a 2d numpy array
    '''
    h, w, _ = image.shape
    new_h = h // 2
    new_w = w // 2

    for i in range(new_h):
      for j in range(new_w):
        im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
        yield im_region, i, j

  def forward(self, input):
    '''
    Performs a forward pass of the maxpool layer using the given input.
    Returns a 3d numpy array with dimensions (h / 2, w / 2, num_filters).
    - input is a 3d numpy array with dimensions (h, w, num_filters)
    '''
    self.last_input = input

    h, w, num_filters = input.shape
    output = np.zeros((h // 2, w // 2, num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.amax(im_region, axis=(0, 1))

    return output

  def backprop(self, d_L_d_out):
    '''
    Performs a backward pass of the maxpool layer.
    Returns the loss gradient for this layer's inputs.
    - d_L_d_out is the loss gradient for this layer's outputs.
    '''
    d_L_d_input = np.zeros(self.last_input.shape)

    for im_region, i, j in self.iterate_regions(self.last_input):
      h, w, f = im_region.shape
      amax = np.amax(im_region, axis=(0, 1))

      for i2 in range(h):
        for j2 in range(w):
          for f2 in range(f):
            # If this pixel was the max value, copy the gradient to it.
            if im_region[i2, j2, f2] == amax[f2]:
              d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]

    return d_L_d_input

In [186]:
import numpy as np

class Softmax:
  # A standard fully-connected layer with softmax activation.

  def __init__(self, input_len, nodes):
    # We divide by input_len to reduce the variance of our initial values
    self.weights = np.random.randn(input_len, nodes) / input_len
    self.biases = np.zeros(nodes)

  def forward(self, input):
    '''
    Performs a forward pass of the softmax layer using the given input.
    Returns a 1d numpy array containing the respective probability values.
    - input can be any array with any dimensions.
    '''
    self.last_input_shape = input.shape

    input = input.flatten()
    self.last_input = input

    input_len, nodes = self.weights.shape

    totals = np.dot(input, self.weights) + self.biases
    self.last_totals = totals

    exp = np.exp(totals)
    return exp / np.sum(exp, axis=0)

  def backprop(self, d_L_d_out, learn_rate):
    '''
    Performs a backward pass of the softmax layer.
    Returns the loss gradient for this layer's inputs.
    - d_L_d_out is the loss gradient for this layer's outputs.
    - learn_rate is a float.
    '''
    # We know only 1 element of d_L_d_out will be nonzero
    for i, gradient in enumerate(d_L_d_out):
      if gradient == 0:
        continue

      # e^totals
      t_exp = np.exp(self.last_totals)

      # Sum of all e^totals
      S = np.sum(t_exp)

      # Gradients of out[i] against totals
      d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
      d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)

      # Gradients of totals against weights/biases/input
      d_t_d_w = self.last_input
      d_t_d_b = 1
      d_t_d_inputs = self.weights

      # Gradients of loss against totals
      d_L_d_t = gradient * d_out_d_t

      # Gradients of loss against weights/biases/input
      d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
      d_L_d_b = d_L_d_t * d_t_d_b
      d_L_d_inputs = d_t_d_inputs @ d_L_d_t

      # Update weights / biases
      self.weights -= learn_rate * d_L_d_w
      self.biases -= learn_rate * d_L_d_b

      return d_L_d_inputs.reshape(self.last_input_shape)

## TEST IT

In [59]:


def forward(image, label):
  '''
  Completes a forward pass of the CNN and calculates the accuracy and
  cross-entropy loss.
  - image is a 2d numpy array
  - label is a digit
  '''
  # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
  # to work with. This is standard practice.
  out = conv.forward((image / 255) - 0.5)
  out = pool.forward(out)
  out = softmax.forward(out)

  # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
  loss = -np.log(out[label])
  acc = 1 if np.argmax(out) == label else 0

  return out, loss, acc

def train(im, label, lr=.005):
  '''
  Completes a full training step on the given image and label.
  Returns the cross-entropy loss and accuracy.
  - image is a 2d numpy array
  - label is a digit
  - lr is the learning rate
  '''
  # Forward
  out, loss, acc = forward(im, label)

  # Calculate initial gradient
  gradient = np.zeros(10)
  gradient[label] = -1 / out[label]

  # Backprop
  gradient = softmax.backprop(gradient, lr)
  gradient = pool.backprop(gradient)
  gradient = conv.backprop(gradient, lr)

  return loss, acc

In [62]:
import urllib.request
from PIL import Image
  
x = str(0) 

IMAGE_PATH = r"/Users/Josh/Desktop/CS 6375 - ML/project/images/"
BASE_PATH: str = "https://personal.utdallas.edu/~kml190002/project-cs6375/train/"

for x in range(5):
    x = str(x)
    urllib.request.urlretrieve( BASE_PATH + x + '.jpg', x + ".jpg")
    img = Image.open(x+".jpg")
    #print(x+".jpg")
    img.save(IMAGE_PATH + x + ".jpeg")
    #img.show()

In [71]:
# read in images
image_list = []

# specify the directory
directory = Path("/Users/Josh/Desktop/CS 6375 - ML/project/images")

# get the list of all files in the directory
image_list = [file for file in directory.iterdir() if file.is_file()]

In [12]:
labels = pd.read_csv("/Users/Josh/Desktop/CS 6375 - ML/project/train.csv")
label_dict = {}
for idx, row in labels.iterrows():
    label_dict.update({row["image_name"]: row["label"]})

In [44]:
# get label
label_dict[image_list[1].stem + ".jpg"]

2

In [74]:
grayImage = Image.open(image_list[0]).convert('L')
imageArray = np.array(grayImage)

print(imageArray.shape)
   
conv = Conv3x3(8)
output = conv.forward(imageArray)
print(output.shape)

pool = MaxPool2()
output = pool.forward(output)
print(output.shape)

softmax = Softmax(output.shape[0]**2 * 8, 6)
output = softmax.forward(output)
print(output.shape)


(150, 150)
(148, 148, 8)
(74, 74, 8)
(6,)


In [252]:
class CNN:
    def __init__(self, image_list: List[Path], labels):
        self.image_list = image_list        
        self.labels = labels
        self.softmax: Softmax = None
        self.images = [Image.open(im) for im in self.image_list]
        self.layers = []

    def addLayer(self, conv: Conv3x3, pool: MaxPool2):
        self.layers.append([conv, pool])

    def numLayers(self):
        return len(self.layers)
    
    def forward(self, layer, image, label= 0, pred = False):
        '''
        Completes a forward pass of the CNN and calculates the accuracy and
        cross-entropy loss.
        - image is a 2d numpy array
        - label is a digit
        '''
        # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
        # to work with. This is standard practice.
        out = layer[0].forward((image / 255) - 0.5)
        out = layer[1].forward(out)

        # if pred == False:
        #     # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
        #     loss = -np.log(out[label])
        #     acc = 1 if np.argmax(out) == label else 0
        # else:
        #     loss = None
        #     acc = None
        loss = None
        acc = None

        return out, loss, acc

    def train(self, lr = 0.005, it = 10):
        """Trains model for each layer on data

        For each iteration, we take each image, and process forward on
        all layers(conv + maxpool...)
        Then when we're done with that, we get the softmax
        Now we can do backprop, but this also needs loop over the layers(conv + maxpool...)
        """
        # need to softcode number of labels
        gradient = np.zeros(6)

        for i in range(it):
            print(i)
            for im in self.image_list:
                label = self.labels[im.stem + ".jpg"]
                input = np.array(Image.open(im))
                
                # this should only go through conv and maxpool. softmax should happen after
                for l in self.layers:
                    out, loss, acc = self.forward(layer = l, image = input, label = label, pred = False)
                    # first input should be original image, then for every additional pass, the input
                    # should be the previous layer's output
                    input = out

                print(out.shape)
                """If first iteration, then create a softmax class
                    Args: 
                        out.shape[0]**2 * out.shape[2] => the previous output's image MASS.
                            This is basically how many nodes there should be in the softmax

                        6 => # of classes to look out for
                """
                if i == 0:
                    self.softmax = Softmax(out.shape[0]**2 * out.shape[2] , 6)
                
                # only run softmax AFTER all the layers full of conv and maxpool
                out = self.softmax.forward(out)
                
                # not sure what to do with this
                gradient[label] = -1 / out[label]

                # Backprop
                gradient = softmax.backprop(gradient, lr)
                for l in self.layers:
                    gradient = pool.backprop(gradient)
                    gradient = conv.backprop(gradient, lr)
            

    def predict(self, image) -> int:
        for l in self.layers:
            out, loss, acc = forward(layer = l, image = image, label = 0, pred = True)

        prob = self.softmax.forward(out)
        pred = np.argmax(prob)

        return pred
        

In [257]:
cnn = CNN(image_list, label_dict)

cnn.addLayer(Conv3x3(8), MaxPool2())
cnn.addLayer(Conv3x3(8), MaxPool2())

### TODO: Figure out softmax layer, then test for backprop

I tried to make this as general as possible, so it __should__ be able to have different number of filters for the different layers....

Worth noting that the shape of output after all layers forward pass corresponds to the number of added layers. Play around with it!

In [258]:
cnn.train()

0
(36, 36, 3)


TypeError: 'NoneType' object is not subscriptable

In [215]:
input = np.random.rand(3,3, 3)

np.sum(input * np.random.rand(3,3))

10.411199351448849

In [79]:
input = np.random.rand(10, 10, 3)

print(input.shape[2])

3
